In [ ]:
import numpy as np
import pandas as pd

size = 5000
df = pd.DataFrame(columns=['Amount', 'Means', 'Business'])
df['Amount'] = np.random.choice(np.arange(1,10001), size=size, p=np.append(np.append(np.repeat(0.5/2000, 2000), np.repeat(0.4/5000, 5000)), np.repeat(0.1/3000, 3000)))
df['Means'] = np.random.choice(["Invoice", "Cash", "Card"], size=size, p=[0.50, 0.40, 0.10])
df['Business'] = np.random.choice([True, False], size=size, p=[0.70, 0.30])

df

,Amount,Means,Business
0,8436,Card,True
1,110,Cash,False
2,1771,Invoice,True
3,4716,Invoice,False
4,507,Cash,False
...,...,...,...
4995,325,Invoice,True
4996,1804,Invoice,False
4997,1937,Cash,True
4998,1165,Invoice,True


In [ ]:
import requests
import json

url = "https://digibp.herokuapp.com/engine-rest/process-definition/key/Synthetic_Data_Gen/tenant-id/showcase/start"

collection = ""

for index, row in df.iterrows():

    if collection:
        collection = collection + ","

    collection = collection + json.dumps({
        "amount": row['Amount'],
        "means": row['Means'],
        "business": row['Business']
    })

payload = json.dumps({
  "variables": {
    "collection": {
      "value": "{\"collection\":["+collection+"]}",
      "type": "json"
    },
    "decision": {
      "value": "payment_decision",
      "type": "String"
    }
  }
})
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"links":[{"method":"GET","href":"https://digibp.herokuapp.com/engine-rest/process-instance/ec28ded8-342c-11ed-a639-ee4a36124437","rel":"self"}],"id":"ec28ded8-342c-11ed-a639-ee4a36124437","definitionId":"Synthetic_Data_Gen:2:373852d4-341c-11ed-a639-ee4a36124437","businessKey":null,"caseInstanceId":null,"ended":false,"suspended":false,"tenantId":"showcase"}


In [ ]:
import httpimport
with httpimport.github_repo('DigiBP', 'digibp-camunda-external-python-task', 'cam'): import cam

worker = cam.Client("https://digibp.herokuapp.com/engine-rest")

response = worker.subscribe("synthetic_data_gen_done", None, "showcase")

taskid = str(response[0]['id'])

data  = json.loads(response[0]['variables']['combinedResult']['value'])['values']

df = df.assign(decision=data)

df.to_csv('payment_decision_file.csv', index=False)

worker.complete(taskid)

df

polling subscription: synthetic_data_gen_done
200
polling subscription: synthetic_data_gen_done
200
polling subscription: synthetic_data_gen_done
200
polling subscription: synthetic_data_gen_done
200
polling subscription: synthetic_data_gen_done
200
polling subscription: synthetic_data_gen_done
200
polling subscription: synthetic_data_gen_done
200
polling subscription: synthetic_data_gen_done
200

204


,Amount,Means,Business,decision
0,6470,Invoice,False,verify
1,6816,Card,True,verify
2,1913,Cash,False,reject
3,948,Invoice,True,approve
4,1858,Invoice,False,verify
...,...,...,...,...
4995,2787,Invoice,False,verify
4996,4239,Cash,False,reject
4997,6422,Card,True,verify
4998,1452,Cash,True,reject


In [ ]:
import numpy as np
import pandas as pd

noise_size = int(size*0.01)
df_noise = pd.DataFrame()
df_noise['index'] = np.random.choice(noise_size, size=noise_size)
df_noise['decision'] = np.random.choice(["approve", "verify", "reject"], size=noise_size, p=[0.50, 0.40, 0.10])

for index, row in df_noise.iterrows():
    df.loc[row['index'],'decision'] = row['decision']

df

,Amount,Means,Business,decision
0,6470,Invoice,False,verify
1,6816,Card,True,verify
2,1913,Cash,False,approve
3,948,Invoice,True,verify
4,1858,Invoice,False,verify
...,...,...,...,...
4995,2787,Invoice,False,verify
4996,4239,Cash,False,reject
4997,6422,Card,True,verify
4998,1452,Cash,True,reject


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3cfa0341-d9ff-404f-9c44-bcc446a7617d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>